In [1]:
import os
import numpy as np
import json
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer, Flatten
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model

In [2]:
# Get the current working directory
CURRENT_DIRECTORY = os.getcwd()
# Get the parent directory
PARENT_DIRECTORY = os.path.dirname(CURRENT_DIRECTORY)
PARENT_DIRECTORY = os.path.dirname(PARENT_DIRECTORY)

# Open the config file and load its content into a dictionary
config_file = open(PARENT_DIRECTORY + '\\config\\config.json')
CONFIG_DATA = json.load(config_file)

# Close the file after loading the data
config_file.close()

In [3]:
WINDOW_SIZE = CONFIG_DATA['WINDOW_SIZE']
NUMBER_OF_FEATURES = CONFIG_DATA["NUMBER_OF_FEATURES"]
NUM_EPOCHS = CONFIG_DATA["NUMBER_OF_EPOCHS"]
CLASSICATIONS = CONFIG_DATA['CLASSES_MOTIONTYPE']

ALL_X_TRAINING_DATA_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\training\\ALL-X-TRAIN-CLASSIFY-MOTIONTYPE-LSIDESTEPS-V4.npy'
ALL_Y_TRAINING_DATA_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\training\\ALL-Y-TRAIN-CLASSIFY-MOTIONTYPE-LSIDESTEPS-V4.npy'

ALL_X_TEST_DATA_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\test\\ALL-X-TEST-CLASSIFY-MOTIONTYPE-LSIDESTEPS-V4.npy'
ALL_Y_TEST_DATA_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\test\\ALL-Y-TEST-CLASSIFY-MOTIONTYPE-LSIDESTEPS-V4.npy'

MOTION_MODEL_SAVE_PATH =  PARENT_DIRECTORY + '\\NeuralNetwork\\models\\standard-backprop-motiontype-lsidesteps-4.h5'

In [4]:
ALL_X_TRAIN_DATA = np.load(ALL_X_TRAINING_DATA_PATH)
ALL_Y_TRAIN_DATA = np.load(ALL_Y_TRAINING_DATA_PATH)

ALL_X_TEST_DATA = np.load(ALL_X_TEST_DATA_PATH)
ALL_Y_TEST_DATA = np.load(ALL_Y_TEST_DATA_PATH)

ALL_X_TRAIN_DATA = ALL_X_TRAIN_DATA[:, [1, 4, 5]]
ALL_X_TEST_DATA = ALL_X_TEST_DATA[:, [1, 4, 5]]

ALL_X_COMBINED_DATA = np.concatenate((ALL_X_TRAIN_DATA, ALL_X_TEST_DATA))
ALL_Y_COMBINED_DATA = np.concatenate((ALL_Y_TRAIN_DATA, ALL_Y_TEST_DATA))


ALL_X_TRAIN_DATA, ALL_X_TEST_DATA, ALL_Y_TRAIN_DATA, ALL_Y_TEST_DATA = train_test_split(ALL_X_COMBINED_DATA, ALL_Y_COMBINED_DATA, test_size=0.2, random_state=42)


print(ALL_X_TRAIN_DATA[:10])
print(ALL_Y_TRAIN_DATA[:10])

print(ALL_X_TRAIN_DATA.shape)
print(ALL_Y_TRAIN_DATA.shape)

print(ALL_X_TEST_DATA.shape)
print(ALL_Y_TEST_DATA.shape)

[[-17.   3.  -5.]
 [-15.   5.  10.]
 [-10.   5.   7.]
 [-10.   3.   0.]
 [-15.   7.  17.]
 [-22.  -1.  -8.]
 [  0.   5.  11.]
 [-10.   5.  18.]
 [-14.   8.  10.]
 [-12.   5.  -1.]]
[0 0 1 1 0 0 1 0 0 0]
(86005, 3)
(86005,)
(21502, 3)
(21502,)


In [5]:
# Generate random permutation indices
random_indices_1 = np.random.permutation(len(ALL_X_TRAIN_DATA))


# Shuffle the input features and target labels using the random indices
shuffled_X_TRAIN_DATA = ALL_X_TRAIN_DATA[random_indices_1]
shuffled_Y_TRAIN_DATA = ALL_Y_TRAIN_DATA[random_indices_1]

In [6]:
# Generate random permutation indices
random_indices_2 = np.random.permutation(len(ALL_X_TEST_DATA))

# Shuffle the input features and target labels using the random indices
shuffled_X_TEST_DATA = ALL_X_TEST_DATA[random_indices_2]
shuffled_Y_TEST_DATA = ALL_Y_TEST_DATA[random_indices_2]

In [7]:
output_activation_function = 'sigmoid' if len(CLASSICATIONS) == 2 else 'softmax'
loss_algorithm = 'binary_crossentropy' if len(CLASSICATIONS) == 2 else 'sparse_categorical_crossentropy'
output_size = 1 if len(CLASSICATIONS) == 2 else len(CLASSICATIONS)

In [8]:
model2_motion = Sequential()
model2_motion.add(InputLayer((3)))
model2_motion.add(Dense(8, activation='relu'))
model2_motion.add(Dense(4, activation='relu'))
model2_motion.add(Dense(output_size, activation=output_activation_function))

model2_motion.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 32        
                                                                 
 dense_1 (Dense)             (None, 4)                 36        
                                                                 
 dense_2 (Dense)             (None, 1)                 5         
                                                                 
Total params: 73
Trainable params: 73
Non-trainable params: 0
_________________________________________________________________


In [9]:
# Compile the model
model2_motion.compile(optimizer='adam', loss=loss_algorithm, metrics=['acc'])

history2_motion = model2_motion.fit(shuffled_X_TRAIN_DATA, shuffled_Y_TRAIN_DATA, validation_data=(shuffled_X_TEST_DATA, shuffled_Y_TEST_DATA), epochs=NUM_EPOCHS)

# Save the model to a file
model2_motion.save(MOTION_MODEL_SAVE_PATH)

Epoch 1/12
2688/2688 [==============================] - 5s 2ms/step - loss: 0.6760 - acc: 0.6078 - val_loss: 0.6629 - val_acc: 0.6217
Epoch 2/12
2688/2688 [==============================] - 4s 1ms/step - loss: 0.4725 - acc: 0.7930 - val_loss: 0.3301 - val_acc: 0.8721
Epoch 3/12
2688/2688 [==============================] - 4s 2ms/step - loss: 0.3134 - acc: 0.8766 - val_loss: 0.2951 - val_acc: 0.8802
Epoch 4/12
2688/2688 [==============================] - 4s 1ms/step - loss: 0.2972 - acc: 0.8781 - val_loss: 0.2869 - val_acc: 0.8775
Epoch 5/12
2688/2688 [==============================] - 4s 1ms/step - loss: 0.2923 - acc: 0.8793 - val_loss: 0.2850 - val_acc: 0.8767
Epoch 6/12
2688/2688 [==============================] - 4s 2ms/step - loss: 0.2891 - acc: 0.8792 - val_loss: 0.2850 - val_acc: 0.8838
Epoch 7/12
2688/2688 [==============================] - 4s 1ms/step - loss: 0.2876 - acc: 0.8796 - val_loss: 0.2815 - val_acc: 0.8752
Epoch 8/12
2688/2688 [==============================] - 4s 2ms

In [10]:
predict_motiontype_lsidesteps = load_model(PARENT_DIRECTORY + '\\NeuralNetwork\\models\\standard-backprop-motiontype-lsidesteps-2.h5')

In [11]:
predict_motiontype_lsidesteps.predict(np.array([[11, 30, -11]]))

1/1 [==============================] - 0s 75ms/step


array([[0.22292179]], dtype=float32)